# SCATTERING GCN: Overcoming Oversmoothness in Graph Convolutional Networks

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.9.0+cu111.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-1.9.0+cu111.html
     |████████████████████████████████| 10.4 MB 8.7 MB/s 
Looking in links: https://data.pyg.org/whl/torch-1.9.0+cu111.html
     |████████████████████████████████| 3.7 MB 7.2 MB/s 
     |████████████████████████████████| 370 kB 10.0 MB/s 
     |████████████████████████████████| 482 kB 58.3 MB/s 
     |████████████████████████████████| 41 kB 722 kB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.3-py3-none-any.whl size=581968 sha256=97de7bf26f526151b77094bbdc12f3c5a3896a1e0a4e22bb4a6cdd436be7b6d3
  Stored in directory: /root/.cache/pip/wheels/c3/2a/58/87ce0508964d4def1aafb92750c4f3ac77038efd1b9a89dcf5
Successfully built torch-geometric


In [ ]:
import torch
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch.nn as nn
from torch.nn.modules.module import Module
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR,StepLR

import numpy as np
import pandas as pd
import pickle as pkl
import sys
import networkx as nx
import scipy.sparse as sp
import math
import matplotlib.pyplot as plt
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Utils functions: normalization, preprocessing and accuracy

In [ ]:
def normalize_adjacency_matrix(A, I):
  """
  Creating a normalized adjacency matrix with self loops.
  :param A: Sparse adjacency matrix.
  :param I: Identity matrix.
  :return A_tile_hat: Normalized adjacency matrix."""
  
  A_tilde = A + I
  degrees = A_tilde.sum(axis=0)[0].tolist()
  D = sp.diags(degrees, [0])
  D = D.power(-0.5)
  A_tilde_hat = D.dot(A_tilde).dot(D)
  return A_tilde_hat

def normalize(mx):
  """Row-normalize sparse matrix ---> Node features"""
  rowsum = np.array(mx.sum(1))
  r_inv = np.power(rowsum, -1).flatten()
  r_inv[np.isinf(r_inv)] = 0.
  r_mat_inv = sp.diags(r_inv)
  mx = r_mat_inv.dot(mx)
  return mx

def normalizemx(mx):
  """Normalization for Scattering GCN"""
  degrees = mx.sum(axis=0)[0].tolist()
  #    print(degrees)
  D = sp.diags(degrees, [0])
  D = D.power(-1)
  mx = mx.dot(D)
  return mx


def scattering1st(spmx,order):

  I_n = sp.eye(spmx.shape[0])
  adj_sct = 0.5*(spmx+I_n) # P = 1/2 * (I + WD^-1)
  adj_power = adj_sct
  adj_power = sparse_mx_to_torch_sparse_tensor(adj_power).cuda()
  adj_sct = sparse_mx_to_torch_sparse_tensor(adj_sct).cuda()
  I_n = sparse_mx_to_torch_sparse_tensor(I_n)
  if order>1:
    for i in range(order-1):
      # Generating P^(2^(k-1))
      adj_power = torch.spmm(adj_power,adj_sct.to_dense())
      print('Generating SCT')
    # Generating. final scattering of order K -> (I - P^(2^(k-1))) * P^(2^(k-1))
    adj_int = torch.spmm((adj_power-I_n.cuda()),adj_power)
  else:
    # Generating. final scattering of order K -> (I - P^(2^(k-1))) * P^(2^(k-1))
    adj_int = torch.spmm((adj_power-I_n.cuda()),adj_power.to_dense())
  return adj_int


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
  """Convert a scipy sparse matrix to a torch sparse tensor."""
  sparse_mx = sparse_mx.tocoo().astype(np.float32)
  indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
  values = torch.from_numpy(sparse_mx.data)
  shape = torch.Size(sparse_mx.shape)
  return torch.sparse.FloatTensor(indices, values, shape)


def parse_index_file(filename):
  #Parse index file.
  index = []
  for line in open(filename):
      index.append(int(line.strip()))
  return index

def accuracy(output, labels):
  preds = output.max(1)[1].type_as(labels)
  correct = preds.eq(labels).double()
  correct = correct.sum()
  return correct / len(labels)

# Preprocessing: Importing datasets

Importing the datasets, split into training, validation and testing, getting the adjacency matrix, the scattering matrices, features matrix, index of nodes.

In [ ]:
def load_citation(dataset_str="pubmed", normalization="AugNormAdj", cuda=True):
  """  
  Load Citation Networks Datasets.
  """
  names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
  objects = []
  for i in range(len(names)):
    with open("/content/drive/MyDrive/THESIS/Databases/data/ind.{}.{}".format(dataset_str.lower(), names[i]), 'rb') as f:
      if sys.version_info > (3, 0):
          objects.append(pkl.load(f, encoding='latin1'))
      else:
          objects.append(pkl.load(f))

  x, y, tx, ty, allx, ally, graph = tuple(objects)
  test_idx_reorder = parse_index_file("/content/drive/MyDrive/THESIS/Databases/data/ind.{}.test.index".format(dataset_str))
  test_idx_range = np.sort(test_idx_reorder)

  if dataset_str == 'citeseer':
    # Fix citeseer dataset (there are some isolated nodes in the graph)
    # Find isolated nodes, add them as zero-vecs into the right position
    test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
    tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
    tx_extended[test_idx_range-min(test_idx_range), :] = tx
    tx = tx_extended
    ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
    ty_extended[test_idx_range-min(test_idx_range), :] = ty
    ty = ty_extended

  features = sp.vstack((allx, tx)).tolil()
  features[test_idx_reorder, :] = features[test_idx_range, :]
  adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))
  adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
  labels = np.vstack((ally, ty))
  labels[test_idx_reorder, :] = labels[test_idx_range, :]


  idx_test = test_idx_range.tolist()
  idx_train = range(len(y))
  idx_val = range(len(y), len(y)+500)

  #   take from https://github.com/tkipf/pygcn/blob/master/pygcn/utils.py
  #    idx_train = range(140)
  #    idx_val = range(200, 500)
  #    idx_test = range(500, 1500)


  labels = torch.LongTensor(labels)
  labels = torch.max(labels, dim=1)[1]
  idx_train = torch.LongTensor(idx_train)
  idx_val = torch.LongTensor(idx_val)
  idx_test = torch.LongTensor(idx_test)

  features = normalize(features)
  A_tilde = normalize_adjacency_matrix(adj,sp.eye(adj.shape[0]))
  adj = normalizemx(adj)
  features = torch.FloatTensor(np.array(features.todense()))
  print('Loading')
  adj_sct1 = scattering1st(adj,1) ## psi_1 = P(I-P)
  adj_sct2 = scattering1st(adj,2) # psi_2 = P^2(I-P^2)
  adj_sct4 = scattering1st(adj,4) # psi_3 = P^4(I-P^4)
  adj = sparse_mx_to_torch_sparse_tensor(adj)
  A_tilde = sparse_mx_to_torch_sparse_tensor(A_tilde)
  return adj,A_tilde,adj_sct1,adj_sct2,adj_sct4,features, labels, idx_train, idx_val, idx_test


In [ ]:
adj,A_tilde,adj_sct1,adj_sct2,adj_sct4,features, labels, idx_train, idx_val, idx_test = load_citation()

UnpicklingError: ignored

# MODELS

First the convolutional structure is defined to finally being called in a nn Module. 

In [ ]:
class GC_withres(Module):
  """
  res conv
  """
  def __init__(self, in_features, out_features,smooth,bias=True):
    super(GC_withres, self).__init__()
    self.in_features = in_features
    self.out_features = out_features
    self.smooth = smooth
    self.weight = Parameter(torch.FloatTensor(in_features, out_features))
    if bias:
        self.bias = Parameter(torch.FloatTensor(out_features))
    else:
        self.register_parameter('bias', None)
    self.reset_parameters()
  def reset_parameters(self):
    stdv = 1. / math.sqrt(self.weight.size(1))
    self.weight.data.uniform_(-stdv, stdv)
    if self.bias is not None:
        self.bias.data.uniform_(-stdv, stdv)
  def forward(self, input, adj):
    print('adj', adj.shape)
    print('input', input.shape)
    # adj is extracted from the graph structure
    support = torch.mm(input, self.weight)
    I_n = sp.eye(adj.shape[0])
    I_n = sparse_mx_to_torch_sparse_tensor(I_n).cuda()
    output = torch.spmm((I_n+self.smooth*adj)/(1+self.smooth), support)
    if self.bias is not None:
        return output + self.bias
    else:
        return output


class NGCN(Module):
  """
  Bandpass model, consider 3 Lap matrix
  """
  def __init__(self, in_features,med_f0,med_f1,med_f2,med_f3,med_f4,bias=True):
    super(NGCN, self).__init__()
    self.in_features = in_features
    self.med_f0 = med_f0
    self.med_f1 = med_f1
    self.med_f2 = med_f2
    self.med_f3 = med_f3
    self.med_f4 = med_f4

    self.weight0 = Parameter(torch.FloatTensor(in_features, med_f0))
    self.weight1 = Parameter(torch.FloatTensor(in_features, med_f1))
    self.weight2 = Parameter(torch.FloatTensor(in_features, med_f2))
    self.weight3 = Parameter(torch.FloatTensor(in_features, med_f3))
    self.weight4 = Parameter(torch.FloatTensor(in_features, med_f4))


    #self.weight = Parameter(torch.FloatTensor((med_f0+med_f1+med_f2), out_features))

    if bias:
        self.bias1 = Parameter(torch.FloatTensor(med_f1))
        self.bias0 = Parameter(torch.FloatTensor(med_f0))
        self.bias2 = Parameter(torch.FloatTensor(med_f2))
        self.bias3 = Parameter(torch.FloatTensor(med_f3))
        self.bias4 = Parameter(torch.FloatTensor(med_f4))

    else:
        self.register_parameter('bias', None)
    self.reset_parameters()
  def reset_parameters(self):
    stdv0 = 1. / math.sqrt(self.weight0.size(1))
    stdv1 = 1. / math.sqrt(self.weight1.size(1))
    stdv2 = 1. / math.sqrt(self.weight2.size(1))

    stdv3 = 1. / math.sqrt(self.weight3.size(1))
    stdv4 = 1. / math.sqrt(self.weight4.size(1))
    torch.nn.init.xavier_uniform(self.weight0)
    torch.nn.init.xavier_uniform(self.weight2)
    torch.nn.init.xavier_uniform(self.weight1)
    torch.nn.init.xavier_uniform(self.weight3)
    torch.nn.init.xavier_uniform(self.weight4)
    if self.bias0 is not None:
        self.bias1.data.uniform_(-stdv1, stdv1)
        self.bias0.data.uniform_(-stdv0, stdv0)
        self.bias2.data.uniform_(-stdv2, stdv2)

        self.bias3.data.uniform_(-stdv3, stdv3)
        self.bias4.data.uniform_(-stdv4, stdv4)

  def forward(self, input, adj,A_tilde,s1_sct,s2_sct,s3_sct,adj_sct_o1,adj_sct_o2):
    # adj is extracted from the graph structure
    # adj_sct_o1,adj_sct_o2: two scatterng matrix index of different order
    # e.g. adj_sct_o1 = [1,1]--> denotes 1st order, 1 index
    # e.g. adj_sct_o1 = [2,1]--> denotes 2nd order
    # 1_sct,2_sct,3_sct: three first order matrix
    support0 = torch.mm(input, self.weight0)
    output0 = torch.spmm(A_tilde, support0) + self.bias0
    support1 = torch.mm(input, self.weight1)
    output1 = torch.spmm(A_tilde, support1)
    output1 = torch.spmm(A_tilde, output1)+ self.bias1

    support2 = torch.mm(input, self.weight2)
    output2 = torch.spmm(A_tilde, support2)
    output2 = torch.spmm(A_tilde, output2)
    output2 = torch.spmm(A_tilde, output2)+ self.bias2
    support3 = torch.mm(input, self.weight3) 
    if adj_sct_o1[0] == 1:
        if adj_sct_o1[1] == 1:
            output3 = torch.spmm(s1_sct.cuda(), support3)+ self.bias3
        elif adj_sct_o1[1] == 2:
            output3 = torch.spmm(s2_sct.cuda(), support3)+ self.bias3
        elif adj_sct_o1[1] == 3:
            output3 = torch.spmm(s3_sct.cuda(), support3)+ self.bias3
        else:
            print('Please type in the right index!')

    elif adj_sct_o1[0] == 2:
        # second order scatt
        # adj_sct_o1[1] == 1----> psi_2|psi_1 x |
        # adj_sct_o1[1] == 2----> psi_3|psi_1 x |
        # adj_sct_o1[1] == 3----> psi_3|psi_2 x |
        if adj_sct_o1[1] == 1:
            output3 = torch.spmm(s2_sct.cuda(),torch.FloatTensor.abs(torch.spmm(s1_sct.cuda(), support3)))+ self.bias3
        elif adj_sct_o1[1] == 2:
            output3 = torch.spmm(s3_sct.cuda(),torch.FloatTensor.abs(torch.spmm(s1_sct.cuda(), support3)))+ self.bias3
        elif adj_sct_o1[1] == 3:
            output3 = torch.spmm(s3_sct.cuda(),torch.FloatTensor.abs(torch.spmm(s2_sct.cuda(), support3)))+ self.bias3
        else:
            print('Please type in the right index!')
    else:
        print('Please type in the right index!')


    support4 = torch.mm(input, self.weight4)
    if adj_sct_o2[0] == 1:
        if adj_sct_o2[1] == 1:
            output4 = torch.spmm(s1_sct.cuda(), support4)+ self.bias4
        elif adj_sct_o2[1] == 2:
            output4 = torch.spmm(s2_sct.cuda(), support4)+ self.bias4
        elif adj_sct_o2[1] == 3:
            output4 = torch.spmm(s3_sct.cuda(), support4)+ self.bias4
        else:
            print('Please type in the right index!')

    elif adj_sct_o2[0] == 2:
        # second order scatt
        # adj_sct_o1[1] == 1----> psi_2|psi_1 x |
        # adj_sct_o1[1] == 2----> psi_3|psi_1 x |
        # adj_sct_o1[1] == 3----> psi_3|psi_2 x |
        if adj_sct_o2[1] == 1:
            output4 = torch.spmm(s2_sct.cuda(),torch.FloatTensor.abs(torch.spmm(s1_sct.cuda(), support4)))+ self.bias4
        elif adj_sct_o2[1] == 2:
            output4 = torch.spmm(s3_sct.cuda(),torch.FloatTensor.abs(torch.spmm(s1_sct.cuda(), support4)))+ self.bias4
        elif adj_sct_o2[1] == 3:
            output4 = torch.spmm(s3_sct.cuda(),torch.FloatTensor.abs(torch.spmm(s2_sct.cuda(), support4)))+ self.bias4
        else:
            print('Please type in the right index!')
    else:
        print('Please type in the right index!')

    print('output0', output0.shape)
    print('output1', output1.shape)
    print('output2', output2.shape)
    print('output3', output3.shape)
    print('output4', output4.shape)
    support_3hop = torch.cat((output0,output1,output2,output3,output4), 1)
    output_3hop = support_3hop
    if self.bias0 is not None:
        print('output', output_3hop.shape)
        return output_3hop  
        #return output_3hop
    else:
        print('output', output_3hop.shape)
        return output_3hop

In [ ]:
class GCN(nn.Module):
  def __init__(self, nfeat, para3,para4, nclass, dropout,smoo):
    super(GCN, self).__init__()

    self.gc1 = NGCN(nfeat,med_f0=10,med_f1=10,med_f2=10,med_f3=para3,med_f4=para4)
    # self.gc1 = NGCN(nfeat,med_f0=28,med_f1=1,med_f2=1,med_f3=para3,med_f4=para4)
    # self.gc2 = NGCN(30+para3+para4,med_f0=28,med_f1=1,med_f2=1,med_f3=para3,med_f4=para4)
    self.gc11 = GC_withres(30+para3+para4, nclass,smooth=smoo)
    self.dropout = dropout

  def forward(self, x,adj, A_tilde,s1_sct,s2_sct,s3_sct,\
          sct_index1,\
          sct_index2):
    x = torch.FloatTensor.abs_(self.gc1(x,adj,A_tilde,\
            s1_sct,s2_sct,s3_sct,\
            adj_sct_o1 = sct_index1,\
            adj_sct_o2 = sct_index2))**4
    x = F.dropout(x, self.dropout, training=self.training)
    x = self.gc11(x, adj)
    return F.log_softmax(x, dim=1)

# Execution of the overall model

Hyperparameter definition, model instatiated, and training and testing

In [ ]:
torch.manual_seed(42)
hidden1 = 16
hidden2 = 51
epochs = 2
lr = 0.005
smoot= 0.5
dropouts = 0.92
order_1 = 1
sct_inx1 = 1
order_2 = 1
sct_inx2 = 3
weight_decays = 0.05
fastmode = False
cuda = torch.cuda.is_available()

model = GCN(nfeat=features.shape[1],para3=hidden1,para4=hidden2,nclass=labels.max().item() + 1,dropout=dropouts,smoo=smoot)

if cuda:
    model = model.cuda()
    features = features.cuda()
    A_tilde = A_tilde.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

    
optimizer = optim.Adam(model.parameters(),lr=lr)
scheduler = StepLR(optimizer, step_size=50, gamma=0.9)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:75: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:76: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:77: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:78: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [ ]:
acc_val_list = []
def train(epoch):


  global valid_error
  t = time.time()
  model.train()
  optimizer.zero_grad()
  output = model(features,adj,A_tilde,adj_sct1,adj_sct2,adj_sct4,[order_1,sct_inx1],[order_2,sct_inx2])
  loss_train = F.nll_loss(output[idx_train], labels[idx_train])

  regularization_loss = 0
  for param in model.parameters():
      regularization_loss = torch.sum(torch.abs(param))

  loss_train = regularization_loss*weight_decays+loss_train
  acc_train = accuracy(output[idx_train], labels[idx_train])
  loss_train.backward()
  optimizer.step()
  if not fastmode:
      # Evaluate validation set performance separately,
      # deactivates dropout during validation run.
      model.eval()
      output = model(features,adj,A_tilde,adj_sct1,adj_sct2,adj_sct4,[order_1,sct_inx1],[order_2,sct_inx2])
  loss_val = F.nll_loss(output[idx_val], labels[idx_val])
  acc_val = accuracy(output[idx_val], labels[idx_val])
  """print('Epoch: {:04d}'.format(epoch+1),
          'hidden1: {:04d}'.format(hidden1),
          'hidden2: {:04d}'.format(hidden2),
        'loss_train: {:.4f}'.format(loss_train.item()),
        'acc_train: {:.4f}'.format(acc_train.item()),
        'loss_val: {:.4f}'.format(loss_val.item()),
        'acc_val: {:.4f}'.format(acc_val.item()),
        'time: {:.4f}s'.format(time.time() - t))"""
  acc_val_list.append(acc_val.item())
  valid_error = 1.0 - acc_val.item()


def test():
  model.eval()
  output = model(features,adj,A_tilde,adj_sct1,adj_sct2,adj_sct4,[order_1,sct_inx1],[order_2,sct_inx2])
  loss_test = F.nll_loss(output[idx_test], labels[idx_test])
  acc_test = accuracy(output[idx_test], labels[idx_test])
  """print("Test set results:",
        "loss= {:.4f}".format(loss_test.item()),
        "accuracy= {:.4f}".format(acc_test.item()))
  acc_val_list.append(acc_test.item())
  """
# Train model
t_total = time.time()
#from pytorchtools import EarlyStopping

#patience = patience
#early_stopping = EarlyStopping(patience=patience, verbose=True)

for epoch in range(epochs):

  train(epoch)
  scheduler.step()
  #    print(valid_error)
  #    early_stopping(valid_error, model)
  #    if early_stopping.early_stop:
  #        print("Early stopping")
  #        break
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

output0 torch.Size([2708, 10])
output1 torch.Size([2708, 10])
output2 torch.Size([2708, 10])
output3 torch.Size([2708, 16])
output4 torch.Size([2708, 51])
output torch.Size([2708, 97])
adj torch.Size([2708, 2708])
input torch.Size([2708, 97])
output0 torch.Size([2708, 10])
output1 torch.Size([2708, 10])
output2 torch.Size([2708, 10])
output3 torch.Size([2708, 16])
output4 torch.Size([2708, 51])
output torch.Size([2708, 97])
adj torch.Size([2708, 2708])
input torch.Size([2708, 97])
output0 torch.Size([2708, 10])
output1 torch.Size([2708, 10])
output2 torch.Size([2708, 10])
output3 torch.Size([2708, 16])
output4 torch.Size([2708, 51])
output torch.Size([2708, 97])
adj torch.Size([2708, 2708])
input torch.Size([2708, 97])
output0 torch.Size([2708, 10])
output1 torch.Size([2708, 10])
output2 torch.Size([2708, 10])
output3 torch.Size([2708, 16])
output4 torch.Size([2708, 51])
output torch.Size([2708, 97])
adj torch.Size([2708, 2708])
input torch.Size([2708, 97])
Optimization Finished!
Total